# Sentiment Anlysis on 20newsgroups Dataset

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
#load 20_Newsgroups train_data
train_data = fetch_20newsgroups(subset='train',shuffle=True)
test_data = fetch_20newsgroups(subset='test',shuffle=True)

In [3]:
#print target name(sub-folders of train data)
train_data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
#First document
#type(train_data.data)
#type(test_data.data)

In [5]:
#Extracting the features
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
#Create feature vector or numerical feature vector, Each unique word is a featuture
count_vec = CountVectorizer()

In [7]:
X_trian_count = count_vec.fit_transform(train_data.data)
X_test_count = count_vec.transform(test_data.data)

In [8]:
print(X_test_count[:,2])

  (1543, 0)	1
  (3999, 0)	1
  (4428, 0)	1
  (6434, 0)	1


In [9]:
print(X_trian_count[:,2])

  (2064, 0)	1
  (3665, 0)	1
  (4191, 0)	1
  (4452, 0)	1
  (4454, 0)	1
  (5447, 0)	1
  (6518, 0)	1


In [10]:
X_trian_count.shape

(11314, 130107)

In [11]:
len(train_data.data)

11314

In [12]:
#Catogories of each file 
train_data.target

array([7, 4, 4, ..., 3, 1, 8])

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

In [14]:
Tfid_transformer = TfidfTransformer()

In [15]:
X_trian_tfidf = Tfid_transformer.fit_transform(X_trian_count)
X_test_tfidf = Tfid_transformer.transform(X_test_count)

In [16]:
#Sparse matrix of 11314 row and 130107 column
#X_trian_tfidf.shape
X_test_tfidf.shape

(7532, 130107)

# Naive Bayes

In [17]:
from sklearn.naive_bayes import MultinomialNB
#fit(feature vector,categories)
classifier = MultinomialNB().fit(X_trian_tfidf,train_data.target)
type(train_data)

sklearn.utils.Bunch

In [18]:
#Building a NB pipeline
from sklearn.pipeline import Pipeline
text_clf_NB = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()), 
                        ('clf', MultinomialNB())])
text_clf_NB = text_clf_NB.fit(train_data.data, train_data.target)

In [19]:
#Apply prediction on test_data
import numpy as np
test_data = fetch_20newsgroups(subset='test', shuffle=True)
predicted_nb = text_clf_NB.predict(test_data.data)

In [20]:
#Calculate the performance of NB Classifier
np.mean(predicted_nb == test_data.target)

0.7738980350504514

In [21]:
#Actual category of test data
print(test_data.target)
#Predicted categoty of test data
print(predicted_nb)

[ 7  5  0 ...,  9  6 15]
[ 7 11  0 ...,  9  3 15]


# Support Vector Machine(SVM)

In [22]:
from sklearn.linear_model import SGDClassifier 

In [23]:
#Buildling a SVM pipeline
from sklearn.pipeline import Pipeline
text_clf_SVM = Pipeline([('vect', CountVectorizer()), 
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

In [24]:
text_clf_SVM = text_clf_SVM.fit(train_data.data,train_data.target)
predicted_svm = text_clf_SVM.predict(test_data.data)
#Calculate the performance of SVM Classifier
np.mean(predicted_svm == test_data.target)

C:\Users\kiit\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.82381837493361654

In [25]:
type(train_data.data)

list

# K-Nearest Neighbor

In [26]:
from sklearn.neighbors import KNeighborsClassifier

In [27]:
negih = KNeighborsClassifier(n_neighbors = 3)

In [28]:
negih.fit(X_trian_tfidf,train_data.target)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [29]:
predicted_knn = negih.predict(X_test_tfidf)

In [30]:
acc = negih.score(X_test_tfidf,test_data.target)

In [31]:
print(acc)

0.657859798194


# Decision Tree

In [32]:
from sklearn import tree

In [33]:
clf = tree.DecisionTreeClassifier()

In [34]:
clf.fit(X_trian_tfidf,train_data.target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [35]:
predicted_dt = negih.predict(X_test_tfidf)

In [36]:
acc = clf.score(X_test_tfidf,test_data.target)

In [37]:
print(acc)

0.554567180032


# Random forest

In [38]:
from sklearn.ensemble import RandomForestClassifier

In [39]:
classifier = RandomForestClassifier(n_estimators = 42, criterion = 'entropy', random_state = 0)

In [40]:
classifier.fit(X_trian_tfidf,train_data.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=42, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [41]:
predicted_rf = classifier.predict(X_test_tfidf)

In [42]:
acc = classifier.score(X_test_tfidf,test_data.target)

In [43]:
print(acc*100)

55.3903345725
